In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import re
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['575-clean-df-all', '575brands']


In [2]:
df_all = pd.read_csv("../input/575-clean-df-all/df_all_toeknized_filled_na_search_term.csv")

In [3]:
brands = pd.read_csv("../input/575brands/brands.csv",sep = "\t")

In [4]:
brands.head()

,product_uid,brand
0,100001,Simpson Strong-Tie
1,100002,BEHR Premium Textured DeckOver
2,100003,STERLING
3,100004,Grape Solar
4,100005,Delta


In [5]:
dt = df_all[["product_uid","search_term","search_term_tokens","search_term_stemmed"]]

In [6]:
dt.head()

,product_uid,search_term,search_term_tokens,search_term_stemmed
0,100001,angle bracket,angle bracket,angl bracket
1,100001,l bracket,l bracket,l bracket
2,100002,deck over,deck over,deck over
3,100005,rain shower head,rain shower head,rain shower head
4,100005,shower only faucet,shower only faucet,shower onli faucet


In [7]:
dt.shape

(186134, 4)

In [8]:
brands.shape

(86250, 2)

In [9]:
dt = dt.merge(brands, on = "product_uid", how = "left")

In [10]:
dt.shape

(186134, 5)

In [11]:
dt["common_brand"] = "None"

In [12]:
dt.head(20)

,product_uid,search_term,search_term_tokens,search_term_stemmed,brand,common_brand
0,100001,angle bracket,angle bracket,angl bracket,Simpson Strong-Tie,None
1,100001,l bracket,l bracket,l bracket,Simpson Strong-Tie,None
2,100002,deck over,deck over,deck over,BEHR Premium Textured DeckOver,None
3,100005,rain shower head,rain shower head,rain shower head,Delta,None
4,100005,shower only faucet,shower only faucet,shower onli faucet,Delta,None
5,100006,convection otr,convection otr,convect otr,Whirlpool,None
6,100006,microwave over stove,microwave over stove,microwav over stove,Whirlpool,None
7,100006,microwaves,microwaves,microwav,Whirlpool,None
8,100007,emergency light,emergency light,emerg light,Lithonia Lighting,None
9,100009,mdf 3/4,mdf 34,mdf,House of Fara,None


In [13]:
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}

def str_stem(s): 
    if isinstance(s, str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        s = s.replace("  "," ")
        s = s.replace(",","") #could be number / segment later
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = s.replace(" x "," xbi ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        #s = (" ").join([z for z in s.split(" ") if z not in stop_w])
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        #s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        
        s = s.lower()
        s = s.replace("toliet","toilet")
        s = s.replace("airconditioner","air conditioner")
        s = s.replace("vinal","vinyl")
        s = s.replace("vynal","vinyl")
        s = s.replace("skill","skil")
        s = s.replace("snowbl","snow bl")
        s = s.replace("plexigla","plexi gla")
        s = s.replace("rustoleum","rust-oleum")
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless","whirlpool stainless")
        return s
    else:
        return "null"

    
def str_common_word(str1, str2):
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

In [14]:
dt['search_term'] = dt['search_term'].map(lambda x:str_stem(x))
dt['brand'] = dt['brand'].map(lambda x:str_stem(x))

In [15]:
dt.head()

,product_uid,search_term,search_term_tokens,search_term_stemmed,brand,common_brand
0,100001,angle bracket,angle bracket,angl bracket,simpson strong tie,None
1,100001,l bracket,l bracket,l bracket,simpson strong tie,None
2,100002,deck over,deck over,deck over,behr premium textured deckover,None
3,100005,rain shower head,rain shower head,rain shower head,delta,None
4,100005,shower only faucet,shower only faucet,shower onli faucet,delta,None


In [16]:
dt['len_of_brand'] = dt['brand'].map(lambda x:len(x.split())).astype(np.int64)
dt['attr'] = dt['search_term']+"\t"+dt['brand']
dt['word_in_brand'] = dt['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
dt['ratio_brand'] = dt['word_in_brand']/dt['len_of_brand']


In [17]:
dt.head(5)

,product_uid,search_term,search_term_tokens,search_term_stemmed,brand,common_brand,len_of_brand,attr,word_in_brand,ratio_brand
0,100001,angle bracket,angle bracket,angl bracket,simpson strong tie,None,3,angle bracket\tsimpson strong tie,0,0.0
1,100001,l bracket,l bracket,l bracket,simpson strong tie,None,3,l bracket\tsimpson strong tie,0,0.0
2,100002,deck over,deck over,deck over,behr premium textured deckover,None,4,deck over\tbehr premium textured deckover,2,0.5
3,100005,rain shower head,rain shower head,rain shower head,delta,None,1,rain shower head\tdelta,0,0.0
4,100005,shower only faucet,shower only faucet,shower onli faucet,delta,None,1,shower only faucet\tdelta,0,0.0


In [18]:
dt[["word_in_brand","ratio_brand"]].to_csv("feature_brand.csv", index = False)